In [ ]:
!pip install datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
train_data = pd.read_pickle("aggregated_train_df_cleaned.pkl")
val_data = pd.read_pickle("aggregated_val_df_cleaned.pkl")
test_data = pd.read_pickle("aggregated_test_df_cleaned.pkl")

In [ ]:
len(train_data) + len(val_data) + len(test_data)

89809

In [ ]:
def preprocess_patient_data(row):
    gender = "male" if row["gender"] == "M" else "female"
    age = f"{row['age']} years old"
    diagnoses = "none" if not row["diagnoses"] else ", ".join(row["diagnoses"])
    abnormal_labs = "none" if not row["abnormal_labs"] else ", ".join(row["abnormal_labs"])
    formatted_history = (
        f"The patient is a {age} {gender}. "
        f"Diagnoses include {diagnoses}. "
        f"Abnormal lab tests include {abnormal_labs}."
    )
    return formatted_history

# Apply preprocessing
train_data["formatted_history"] = train_data.apply(preprocess_patient_data, axis=1)
val_data["formatted_history"] = val_data.apply(preprocess_patient_data, axis=1)
test_data["formatted_history"] = test_data.apply(preprocess_patient_data, axis=1)

# Extract inputs and labels
train_texts = train_data["formatted_history"].tolist()
train_labels = train_data["is_ckd"].tolist()

val_texts = val_data["formatted_history"].tolist()
val_labels = val_data["is_ckd"].tolist()

test_texts = test_data["formatted_history"].tolist()
test_labels = test_data["is_ckd"].tolist()

In [ ]:
# Load tokenizer
# model_name = "Charangan/MedBERT"
# model_name = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize_texts(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

# Tokenize datasets
train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)
test_encodings = tokenize_texts(test_texts)


tokenizer_config.json:   0%|          | 0.00/417 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})

In [ ]:
import numpy as np
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

class_weights_tensor = torch.tensor(class_weights).to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class_weights_tensor

tensor([0.5627, 4.4898], device='cuda:0', dtype=torch.float64)

In [ ]:
class WeightedLossModel(AutoModelForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels, **kwargs)
        logits = outputs.logits

        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
            loss = loss_fn(logits, labels)
            return torch.nn.functional.loss(outputs.loss, loss)

        return outputs

In [ ]:
model = WeightedLossModel.from_pretrained(model_name, num_labels=2)

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Charangan/MedBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, auc, precision_recall_curve

def compute_metrics(pred):
    """
    Computes evaluation metrics for the classification model.

    Args:
    pred: Predictions and labels from the Trainer.

    Returns:
    Dictionary containing accuracy, precision, recall, F1 score, PR AUC, and ROC AUC.
    """
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)
    probabilities = logits[:, 1]  # Assuming binary classification with probabilities for the positive class

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    roc_auc = roc_auc_score(labels, probabilities)

    # PR AUC (Area Under Precision-Recall Curve)
    precision_curve, recall_curve, _ = precision_recall_curve(labels, probabilities)
    pr_auc = auc(recall_curve, precision_curve)

    return {
        "accuracy": acc,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    metric_for_best_model="roc_auc",
    greater_is_better=True,
    report_to=[]  # Turn off Weights & Biases
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-37-5cbc6386e7a0>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
# Evaluate on test data
results = trainer.evaluate(test_dataset)
print(results)

In [ ]:
import os
import shutil
import zipfile
from peft import PeftModel

def save_trained_model_as_zip(trainer, save_dir="./saved_model", zip_file="./saved_model.zip"):
    """Save the trained model and tokenizer as a zip file."""
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Save the model and tokenizer to the directory
    trainer.model.save_pretrained(save_dir)
    trainer.tokenizer.save_pretrained(save_dir)

    # Create a zip file
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(save_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, save_dir)
                zipf.write(file_path, arcname)

    print(f"Model and tokenizer saved as a zip file to {zip_file}")

# Example usage
save_trained_model_as_zip(trainer)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Model and tokenizer saved as a zip file to ./saved_model.zip
